In [1]:
import time
import sys
import os
path_project = "\\".join(os.getcwd().split("\\")[:-1])
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, path_project)

from hardware.pulser.pulser import PulseGenerator, OutputState, TriggerStart, TriggerRearm, HIGH, LOW, INF
from hardware.laser.laser import LaserControl
from hardware.mw.mwsynthesizer import Synthesizer

from hardware import config_custom as hcf

import nidaqmx
from nidaqmx.constants import TerminalConfiguration, VoltageUnits, Edge, AcquisitionType, READ_ALL_AVAILABLE
# from nidaqmx.constants import LogicFamily 
# TWO_POINT_FIVE_V, THREE_POINT_THREE_V, FIVE_V
# from nidaqmx import stream_readers 
# from nidaqmx.stream_readers import AnalogSingleChannelReader

import numpy as np
from windfreak import SynthHD
from math import floor
import time

In [ ]:
# connect to the digitizer 
from hardware.daq.sidig import FIFO_DataAcquisition
from hardware.daq.sidig import FIFO_DataAcquisition, DCCOUPLE, TERMIN_INPUT_50OHM, TERMIN_INPUT_1MOHM

SIDIG_ADDRESS = "dev/spcm0"
SIDIG_maxsr = 500e6  # Hz
SIDIG_timebase = int(1 / SIDIG_maxsr * 1e9)  # ns
dig=FIFO_DataAcquisition(
                sn_address=SIDIG_ADDRESS
            )

### Pulse streamer connection 

In [ ]:
# connect to the pulse streamer 
pg  = PulseGenerator(ip=hcf.PS_IP, chmap=hcf.PS_chmap) 


### Windfreak setup

In [ ]:
# connect to the windfreak mw driver 
synth = SynthHD('COM4')
synth.init()
# ch = synth[0]  

Connect to Pulse Streamer via JSON-RPC.
IP / Hostname: 192.168.1.26
Pulse Streamer 8/2 firmware: v1.7.2
Client software: v1.7.0


### Laser control

In [ ]:
# laser = LaserControl(hcf.LASER_SN)
# laser.open()

In [ ]:
# current_percent = 75.09
# laser.laser_off()
# laser.set_analog_control_mode("current")
# laser.set_modulation_state("Pulsed")
# laser.set_diode_current(current_percent, save_memory=False)

In [ ]:
sweep_config = {
    # WINDFREAK SETTINGS
    "sweep_start_MHz": 400,
    "sweep_stop_MHz": 600,
    "sweep_step_size_MHz": 25,
    "pause_at_step_us": 1e9,
    "sweep_power": 0,       # dB
    'sweep_direction': 1 ,  # up = 1 down = 0.
    'sweep_type': 0 ,       # 0 = linear 1 = tabular 
    'sweep_only_once': True, # True = 1 rep False = inf reps
    'sweep_all_freq': False, # True = after a trigger sweep through all freq False = after trigger step 1 freq only 
    'trigger_type': 3, # 0 = software 1 = sweep 2 = step 3 = hold all 
    'trigger_polarity': 1, # active low =0 active high = 1,
    # DIGITZER SETTINGS
    'readout_ch': hcf.SIDIG_chmap["apd"],
    'amp_input' : 1000,
    'terminate_input': TERMIN_INPUT_1MOHM,
    'DCCOUPLE': 0,
    'sampling_rate': hcf.SIDIG_maxsr,
    # DIGITIZER collection 
    'num_segment': 8,
    'pretrig_size': 16,
    'posttrig_size': 1000,
    'num_samples_per_segment': None, # assigned in next line 
    'segment_size': 1000,
    # Loop settings
    'run_time': 10, # seconds
}
sweep_config['num_samples_per_segment'] = sweep_config['pretrig_size'] + sweep_config['posttrig_size']

In [9]:
trig_len=10
readout_period=sweep_config['pause_at_step_us']+10000 #TODO: config len
freq_sweep_trig=[(sweep_config['pause_at_step_us'],HIGH),(10000,LOW)] #TODO: can you write this as a 10ns pulse with wait time?
switch_trig=[(sweep_config['pause_at_step_us'],HIGH),(10000,LOW)] #TODO: config len
laser_trig=[(readout_period-3*trig_len,HIGH),(3*trig_len,LOW)] # "always" on, off at end to avoid dig freq filter #TODO: config len
dig_trig=[(trig_len,HIGH),(readout_period/2 - trig_len,LOW)]*2 #TODO: config len


pg.setDigital("WDF", freq_sweep_trig)
# pg.setDigital("switch", switch_trig) # TODO: add channel for the switch
pg.setDigital("laser", laser_trig)
pg.setDigital("sdtrig", dig_trig)

pg.setTrigger(TriggerStart.SOFTWARE, rearm=TriggerRearm.AUTO)
REPEAT_INFINITELY = -1
# pg.stream(n_runs=1)
pg.plotSeq(plot_all=False)


In [5]:
pg.stream(n_runs=100)
pg.startNow()

0

In [ ]:
# step with ext trigger V4 
# yes works by gated on high and low pulse from the pulse streamer 
synth[0].power = -80

ch.write('frequency',sweep_config['sweep_start_MHz'])
ch.write('sweep_freq_low', sweep_config['sweep_start_MHz'])
ch.write('sweep_freq_high', sweep_config['sweep_stop_MHz'])
ch.write('sweep_freq_step', sweep_config['sweep_step_size_MHz'])
ch.write('sweep_time_step',0) #TODO check if this is necessary in stepped version
ch.write('sweep_power_low', sweep_config['sweep_power'])
ch.write('sweep_power_high', sweep_config['sweep_power'])
ch.write('sweep_direction', sweep_config['sweep_direction'])
ch.write('sweep_diff_freq', 0)
ch.write('sweep_diff_meth', 0)
ch.write('sweep_type', sweep_config['sweep_type'])
ch.write('sweep_cont', sweep_config['step_only_once'])
ch.write('sweep_single', sweep_config['sweep_all_freq'])
synth.enable_trigger=sweep_config['trigger_type'] # 0 = software 1 = sweep 2 = step 3 = hold all 
synth.trigger_polarity=sweep_config['trigger_polarity'] # active low =0 active high = 1
synth.trigger_mode="single frequency step"
 
# synth.rf_enable = True # not necessary
# ch.rf_enable = True # not necessary

ch.enable = True 
synth.enable=True
# synth.sweep_enable = True 3 not necessary either 
# ch.sweep_enable = True

### Digitzer config setup

In [ ]:
# To set the configuration, make a dictionary with the key and value
# TODO: config the parameters for the digitizer
dig.reset_param()
dig.assign_param(dict(
             readout_ch=sweep_config['readout_ch'], 
             amp_input=sweep_config['amp_input'], 
             num_segment=sweep_config['num_segment'],
             pretrig_size=sweep_config['pretrig_size'],
             posttrig_size=sweep_config['posttrig_size'],
             segment_size=sweep_config['segment_size'],
             terminate_input=sweep_config['terminate_input'],
             DCCOUPLE = sweep_config['DCCOUPLE'],
             sampling_rate=sweep_config['sampling_rate'],
            #  terminate_input=TERMIN_INPUT_50OHM, #TODO: check this - why is it commented out
             ))

In [ ]:
start_time = time.time()

while (time.time() - start_time) < sweep_config['run_time']:
    print("hello")



    

In [ ]:
# disable all the inst 
dig.stop_card()
ch.enable = False
synth.enable=False
pg.forceFinal()
pg.reset()


In [ ]:
#TODO: add streaming logic for the digitizer 
